In [1]:
import scala.util.Try
import scala.math.max
import org.apache.spark._
import org.apache.log4j._
import org.apache.spark.rdd.RDD
import org.apache.spark.sql.{DataFrame, Dataset, SparkSession}
import org.apache.spark.sql.functions.{concat, lit}
import org.apache.spark.sql.functions.{col, udf}
import org.apache.spark.sql.functions.expr
import org.apache.spark.sql.functions._
import org.apache.spark.SparkContext
import org.apache.spark.SparkContext._
import org.apache.spark.sql.types._
import java.net.URLDecoder

In [2]:
import org.apache.spark.ml.feature.VectorAssembler
import org.apache.spark.ml.linalg.{DenseVector, SparseVector, Vector, Vectors}
import org.apache.spark.ml.evaluation.BinaryClassificationEvaluator
import org.apache.spark.ml.classification.{RandomForestClassificationModel, RandomForestClassifier}
import org.apache.spark.ml.classification.{GBTClassificationModel, GBTClassifier}

In [3]:
val spark = SparkSession.builder().appName("test_lab05").config("spark.master", "yarn").getOrCreate()

spark = org.apache.spark.sql.SparkSession@1bc1b74b


org.apache.spark.sql.SparkSession@1bc1b74b

In [4]:
// считываем исходные данные
val train = spark.read.option("delimiter", ",").option("header", "True").option("encoding","utf-8").csv("/labs/slaba05/lab05_train.csv")

val test = spark.read.option("delimiter", ",").option("header", "True").option("encoding","utf-8").csv("/labs/slaba05/lab05_test.csv")

train = [_c0: string, ID: string ... 115 more fields]
test = [_c0: string, ID: string ... 114 more fields]


[_c0: string, ID: string ... 114 more fields]

In [34]:
// смотрим топ 20 записей
train.show(numRows = 1, truncate = 100, vertical=true)

-RECORD 0--------------------------------------------
 _c0                         | 333149                
 ID                          | 479990                
 CR_PROD_CNT_IL              | 0                     
 AMOUNT_RUB_CLO_PRC          | 0.0                   
 PRC_ACCEPTS_A_EMAIL_LINK    | 0.0                   
 APP_REGISTR_RGN_CODE        | null                  
 PRC_ACCEPTS_A_POS           | 0.0                   
 PRC_ACCEPTS_A_TK            | 0.0                   
 TURNOVER_DYNAMIC_IL_1M      | 0.0                   
 CNT_TRAN_AUT_TENDENCY1M     | null                  
 SUM_TRAN_AUT_TENDENCY1M     | null                  
 AMOUNT_RUB_SUP_PRC          | 0.0                   
 PRC_ACCEPTS_A_AMOBILE       | 0.0                   
 SUM_TRAN_AUT_TENDENCY3M     | null                  
 CLNT_TRUST_RELATION         | null                  
 PRC_ACCEPTS_TK              | 0.0                   
 PRC_ACCEPTS_A_MTP           | 0.0                   
 REST_DYNAMIC_FDEP_1M       

In [6]:
// смотрим структуру датасета train
train.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- ID: string (nullable = true)
 |-- CR_PROD_CNT_IL: string (nullable = true)
 |-- AMOUNT_RUB_CLO_PRC: string (nullable = true)
 |-- PRC_ACCEPTS_A_EMAIL_LINK: string (nullable = true)
 |-- APP_REGISTR_RGN_CODE: string (nullable = true)
 |-- PRC_ACCEPTS_A_POS: string (nullable = true)
 |-- PRC_ACCEPTS_A_TK: string (nullable = true)
 |-- TURNOVER_DYNAMIC_IL_1M: string (nullable = true)
 |-- CNT_TRAN_AUT_TENDENCY1M: string (nullable = true)
 |-- SUM_TRAN_AUT_TENDENCY1M: string (nullable = true)
 |-- AMOUNT_RUB_SUP_PRC: string (nullable = true)
 |-- PRC_ACCEPTS_A_AMOBILE: string (nullable = true)
 |-- SUM_TRAN_AUT_TENDENCY3M: string (nullable = true)
 |-- CLNT_TRUST_RELATION: string (nullable = true)
 |-- PRC_ACCEPTS_TK: string (nullable = true)
 |-- PRC_ACCEPTS_A_MTP: string (nullable = true)
 |-- REST_DYNAMIC_FDEP_1M: string (nullable = true)
 |-- CNT_TRAN_AUT_TENDENCY3M: string (nullable = true)
 |-- CNT_ACCEPTS_TK: string (nullable = true)
 |--

In [38]:
// test.show(numRows = 1, truncate = 100, vertical=true)

Name: Syntax Error.
Message: 
StackTrace: 

In [37]:
// test.printSchema()

Name: Syntax Error.
Message: 
StackTrace: 

In [7]:
// 
var df_train = train.drop("APP_CAR", "APP_COMP_TYPE", "APP_COMP_TYPE", "APP_DRIVING_LICENSE", "APP_EMP_TYPE", 
                      "APP_KIND_OF_PROP_HABITATION", "APP_MARITAL_STATUS", "APP_POSITION_TYPE", "APP_TRAVEL_PASS",
                      "APP_REGISTR_RGN_CODE", "APP_TRAVEL_PASS", 
                      "CLNT_JOB_POSITION_TYPE", "PACK", "_c0",
                     "CLNT_TRUST_RELATION", "CLNT_JOB_POSITION")
var df_test = test.drop("APP_CAR", "APP_COMP_TYPE", "APP_COMP_TYPE", "APP_DRIVING_LICENSE", "APP_EMP_TYPE", 
                      "APP_KIND_OF_PROP_HABITATION", "APP_MARITAL_STATUS", "APP_POSITION_TYPE", "APP_TRAVEL_PASS",
                      "APP_REGISTR_RGN_CODE", "APP_TRAVEL_PASS", 
                      "CLNT_JOB_POSITION_TYPE", "PACK", "_c0",
                     "CLNT_TRUST_RELATION", "CLNT_JOB_POSITION")

df_train = [ID: string, CR_PROD_CNT_IL: string ... 101 more fields]
df_test = [ID: string, CR_PROD_CNT_IL: string ... 100 more fields]


[ID: string, CR_PROD_CNT_IL: string ... 100 more fields]

In [9]:
var feature_list = Array[String]()
for (t <- df_train.columns if (t!="ID"))
{ df_train = df_train.withColumn(t, col(t).cast(FloatType))
        feature_list +:= t}
val b = feature_list.filter(! _.contains("TARGET"))

feature_list = Array(TARGET, LDEAL_ACT_DAYS_PCT_CURR, LDEAL_ACT_DAYS_PCT_TR4, LDEAL_ACT_DAYS_PCT_TR, TURNOVER_DYNAMIC_CC_3M, LDEAL_DELINQ_PER_MAXYWZ, LDEAL_ACT_DAYS_PCT_AAVG, LDEAL_ACT_DAYS_PCT_TR3, MED_DEBT_PRC_YWZ, REST_DYNAMIC_CC_3M, LDEAL_ACT_DAYS_ACC_PCT_AVG, AVG_PCT_DEBT_TO_DEAL_AMT, TURNOVER_DYNAMIC_CC_1M, LDEAL_USED_AMT_AVG_YWZ, REST_DYNAMIC_CC_1M, LDEAL_USED_AMT_AVG_YQZ, TRANS_CNT_TENDENCY3M, MED_DEBT_PRC_YQZ, TRANS_AMOUNT_TENDENCY3M, CLNT_SALARY_VALUE, TURNOVER_DYNAMIC_PAYM_1M, LDEAL_DELINQ_PER_MAXYQZ, TURNOVER_DYNAMIC_PAYM_3M, DEAL_GRACE_DAYS_ACC_MAX, CLNT_SETUP_TENOR, LDEAL_YQZ_PC, MAX_PCLOSE_DATE, TURNOVER_DYNAMIC_CUR_3M, DEAL_GRACE_DAYS_ACC_AVG, CNT_TRAN_SUP_TENDENCY1M, REST_DYNAMIC_PAYM_1M, SUM_TRAN_ATM_TENDENCY1M, DEAL_YWZ_IR_MAX, SUM_TRAN_SUP_TENDENCY1M, ...


Array(TARGET, LDEAL_ACT_DAYS_PCT_CURR, LDEAL_ACT_DAYS_PCT_TR4, LDEAL_ACT_DAYS_PCT_TR, TURNOVER_DYNAMIC_CC_3M, LDEAL_DELINQ_PER_MAXYWZ, LDEAL_ACT_DAYS_PCT_AAVG, LDEAL_ACT_DAYS_PCT_TR3, MED_DEBT_PRC_YWZ, REST_DYNAMIC_CC_3M, LDEAL_ACT_DAYS_ACC_PCT_AVG, AVG_PCT_DEBT_TO_DEAL_AMT, TURNOVER_DYNAMIC_CC_1M, LDEAL_USED_AMT_AVG_YWZ, REST_DYNAMIC_CC_1M, LDEAL_USED_AMT_AVG_YQZ, TRANS_CNT_TENDENCY3M, MED_DEBT_PRC_YQZ, TRANS_AMOUNT_TENDENCY3M, CLNT_SALARY_VALUE, TURNOVER_DYNAMIC_PAYM_1M, LDEAL_DELINQ_PER_MAXYQZ, TURNOVER_DYNAMIC_PAYM_3M, DEAL_GRACE_DAYS_ACC_MAX, CLNT_SETUP_TENOR, LDEAL_YQZ_PC, MAX_PCLOSE_DATE, TURNOVER_DYNAMIC_CUR_3M, DEAL_GRACE_DAYS_ACC_AVG, CNT_TRAN_SUP_TENDENCY1M, REST_DYNAMIC_PAYM_1M, SUM_TRAN_ATM_TENDENCY1M, DEAL_YWZ_IR_MAX, SUM_TRAN_SUP_TENDENCY1M, ...

In [10]:
for (t <- df_test.columns if (t!="ID"))
{ df_test = df_test.withColumn(t, col(t).cast(FloatType))}

In [11]:
df_test = df_test.na.fill(value=0.0)
df_train = df_train.na.fill(value=0.0)

df_test = [ID: string, CR_PROD_CNT_IL: float ... 100 more fields]
df_train = [ID: string, CR_PROD_CNT_IL: float ... 101 more fields]


[ID: string, CR_PROD_CNT_IL: float ... 101 more fields]

In [12]:
val assembler = new VectorAssembler()
                    .setInputCols(b)
                    .setOutputCol("features")

assembler = vecAssembler_6c55c7418807


vecAssembler_6c55c7418807

In [13]:
df_train = df_train.withColumn("TARGET", col("TARGET").cast(IntegerType))

df_train = [ID: string, CR_PROD_CNT_IL: float ... 101 more fields]


[ID: string, CR_PROD_CNT_IL: float ... 101 more fields]

In [14]:
val train_data=assembler.transform(df_train).repartition(30)
val test_data=assembler.transform(df_test).repartition(30)

train_data = [ID: string, CR_PROD_CNT_IL: float ... 102 more fields]
test_data = [ID: string, CR_PROD_CNT_IL: float ... 101 more fields]


[ID: string, CR_PROD_CNT_IL: float ... 101 more fields]

In [15]:
val evaluator = new BinaryClassificationEvaluator()
                        .setLabelCol("TARGET")
                        .setRawPredictionCol("probability")
                        .setMetricName("areaUnderROC")

evaluator = binEval_f709ed236e70


binEval_f709ed236e70

In [16]:
val gbt = new GBTClassifier()
      .setFeaturesCol("features")
      .setLabelCol("TARGET")
      .setMaxDepth(4)
      .setMaxIter(50)

gbt = gbtc_8ce1f65ad789


gbtc_8ce1f65ad789

In [17]:
//обучаем модель
val model_gbt = gbt.fit(train_data)

model_gbt = GBTClassificationModel (uid=gbtc_8ce1f65ad789) with 50 trees


GBTClassificationModel (uid=gbtc_8ce1f65ad789) with 50 trees

In [19]:
val predictions_train = model_gbt.transform(train_data)

predictions_train = [ID: string, CR_PROD_CNT_IL: float ... 105 more fields]


[ID: string, CR_PROD_CNT_IL: float ... 105 more fields]

In [20]:
val ROC_AUC = evaluator.evaluate(predictions_train)

ROC_AUC = 0.829076328679496


0.829076328679496

In [21]:
val predictions_test = model_gbt.transform(test_data)

predictions_test = [ID: string, CR_PROD_CNT_IL: float ... 104 more fields]


[ID: string, CR_PROD_CNT_IL: float ... 104 more fields]

In [22]:
val sparse_to_dense = udf((v: Vector) =>  v.toArray)

sparse_to_dense = UserDefinedFunction(<function1>,ArrayType(DoubleType,false),Some(List(org.apache.spark.ml.linalg.VectorUDT@3bfc3ba7)))


UserDefinedFunction(<function1>,ArrayType(DoubleType,false),Some(List(org.apache.spark.ml.linalg.VectorUDT@3bfc3ba7)))

In [23]:
val to_file = predictions_test.select(col("ID").alias("id"), sparse_to_dense(col("probability")).getItem(1).alias("target")).coalesce(1)

to_file = [id: string, target: double]


[id: string, target: double]

In [24]:
to_file.show()

+------+--------------------+
|    id|              target|
+------+--------------------+
|338743|0.024759072325617026|
|396274|0.038163679377554494|
|317805| 0.02458297763532946|
|398424| 0.02590819013345458|
|373496|  0.3270877621129269|
|209206| 0.24066875334233184|
|319688| 0.06209164699009506|
|345326|0.026610217650548273|
|373023| 0.02671153458371367|
|225004| 0.19473992323704659|
|192599| 0.10599952959487369|
|470522| 0.10196508962487238|
|537732| 0.05514734894227491|
|444496|  0.0409344841767455|
|244935|0.026071574039680634|
|485741| 0.23867742595789254|
|551227| 0.07690448138110895|
|395048| 0.14711423368085386|
|255069| 0.04646592087985679|
|274320| 0.12335631847361006|
+------+--------------------+
only showing top 20 rows



In [44]:
// сохраняю файл результата
// to_file.coalesce(1).write.option("header","true").option("sep","\t").mode("overwrite").csv("lab05.csv")
to_file.write.format("com.databricks.spark.csv").save("lab05.csv")

Name: Syntax Error.
Message: 
StackTrace: 

In [48]:
// org.apache.spark.SparkFiles.get(to_file)
org.apache.spark.SparkFiles.getRootDirectory

/tmp/spark-3322c2d5-2b4e-4973-8e36-ce67a06f838f/userFiles-58b2d8fa-f139-49e7-ab03-447bbdeacdcb

In [43]:
// забираю файл 
import scala.sys.process._

"""hdfs dfs -get hdfs://spark-master-1.newprolab.com:8020/user/aleksandr.balandin/lab05.csv""".!!

lastException: Throwable = null
res77: String = ""


In [49]:
spark.stop()